In [17]:
import os
import numpy as np
import pandas as pd

user_act = pd.read_csv("../data/user_action.csv", sep=',', error_bad_lines=False, encoding='utf-8', header=0)
df1 = user_act.drop(labels=['id',
                            'device_id', 'os', 'os_version', 'version', 'system', 'platform', 'pg_short_url',
                            'log_time', 'cal_dt', 'duration', 'log_id'], axis=1)
df1.loc[((df1.event_id == 254) | (df1.event_id == 248)), 'follow'] = 'thumb'
df1.loc[((df1.event_id == 257) | (df1.event_id == 249)), 'follow'] = 'comment'
df1.loc[((df1.event_id == 258) | (df1.event_id == 256)), 'follow'] = 'forward'
df1.loc[((df1.event_id == 262) | (df1.event_id == 263)), 'follow'] = 'detail'
df1.loc[((df1.event_id == 264) | (df1.event_id == 310)), 'follow'] = 'detail'
df1['follow'] = df1['follow'].fillna(0)
# user_new['follow'] = user_new['follow'].fillna(0)
df1['follow'] = df1['follow'].replace(0, 'neg')
df1['follow'].value_counts()
import json

# df1['content_id'] = df1.loc[(df1.event_id != 221), 'event_data']
df1['event_data'] = df1['event_data'].apply(json.loads)


def process(event_data):
    if not isinstance(event_data, dict):
        # print('list')
        return 0
    if 'content_id' in event_data.keys():
        return event_data['content_id']


df1['event_data'] = df1['event_data'].apply(process)
df1['event_data'] = pd.to_numeric(df1['event_data']).fillna('0').astype('int64')
df1['event_data'].value_counts()
df1['content_id'] = df1['event_data']

df1 = df1.sort_values(by=['user_id', 'created_at'], ascending=True)

/Users/starry/conda/envs/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [9]:

# df1 = df1.head(50000)

In [18]:
df1.head()

,user_id,event_id,event_data,created_at,follow,content_id
11077732,0,35,711680,1635955758202,neg,711680
11088991,0,35,694645,1635958983831,neg,694645
11088992,0,35,711680,1635958983842,neg,711680
11090685,0,35,711948,1635959535768,neg,711948
11090687,0,35,712029,1635959535774,neg,712029


In [60]:
df2 = df1[df1['follow'] != 'neg']
df2[df2['user_id']==1540291594230058]

,user_id,event_id,event_data,created_at,follow,content_id
7435274,1540291594230058,262,694645,1637206898714,detail,694645
7435380,1540291594230058,254,997,1637206931771,thumb,997
7435384,1540291594230058,257,732208,1637206932867,comment,732208
7435385,1540291594230058,262,732208,1637206932874,detail,732208
7435417,1540291594230058,254,0,1637206955479,thumb,0
7435452,1540291594230058,262,731479,1637206961222,detail,731479
7435497,1540291594230058,262,728865,1637206980803,detail,728865
7435787,1540291594230058,254,0,1637207064567,thumb,0
7436050,1540291594230058,254,0,1637207115181,thumb,0
7436627,1540291594230058,262,732005,1637207246186,detail,732005


In [66]:
df_t = df2[df2['user_id']!=0].head(10000)

In [67]:

from functools import reduce

df_t['content_list'] = pd.NA


In [68]:

def count_inter_list(user_id):
    inter = list(df_t.loc[(df_t.user_id == user_id), 'content_id'])
    func = lambda x, y: x if y in x else x + [y]
    list1 = reduce(func, [[], ] + inter)
    content_list = map(lambda x: str(x), list1)
    str1 = ",".join(list(content_list))
    return str1



%time df_t['content_list'] = df_t.apply(lambda row: count_inter_list(row['user_id']), axis=1)
# df4['inter_list'] = count_inter_list(df4['user_id'].values, df4['content_id'].values)

CPU times: user 9.17 s, sys: 40.2 ms, total: 9.21 s
Wall time: 9.26 s


In [58]:
df_t


,user_id,event_id,event_data,created_at,follow,content_id,content_list
5003010,1540291169530038,263,739637,1637978771269,detail,739637,739637
10176143,1540291528300057,262,716856,1636301049165,detail,716856,"716856,718538,716706,726790,732897,733148,7351..."
10269519,1540291528300057,262,718538,1636354084428,detail,718538,"716856,718538,716706,726790,732897,733148,7351..."
10269639,1540291528300057,262,716706,1636354144578,detail,716706,"716856,718538,716706,726790,732897,733148,7351..."
8941458,1540291528300057,262,726790,1636788738222,detail,726790,"716856,718538,716706,726790,732897,733148,7351..."
...,...,...,...,...,...,...,...
2842009,1542120650393517,262,758406,1638698311025,detail,758406,"749645,753973,755646,486917,420594,757459,7557..."
2843388,1542120650393517,262,755689,1638698805325,detail,755689,"749645,753973,755646,486917,420594,757459,7557..."
3012279,1542120650393517,262,763223,1638717116510,detail,763223,"749645,753973,755646,486917,420594,757459,7557..."
3015782,1542120650393517,263,761527,1638717310051,detail,761527,"749645,753973,755646,486917,420594,757459,7557..."


In [65]:
df_t1

,user_id,event_id,event_data,created_at,follow,content_id,rep,content_list
5003010,1540291169530038,263,739637,1637978771269,detail,739637,False,739637
10176143,1540291528300057,262,716856,1636301049165,detail,716856,True,"716856,718538,716706,726790,732897,733148,7351..."
10269519,1540291528300057,262,718538,1636354084428,detail,718538,True,"716856,718538,716706,726790,732897,733148,7351..."
10269639,1540291528300057,262,716706,1636354144578,detail,716706,True,"716856,718538,716706,726790,732897,733148,7351..."
8941458,1540291528300057,262,726790,1636788738222,detail,726790,True,"716856,718538,716706,726790,732897,733148,7351..."
...,...,...,...,...,...,...,...,...
2842009,1542120650393517,262,758406,1638698311025,detail,758406,True,"749645,753973,755646,486917,420594,757459,7557..."
2843388,1542120650393517,262,755689,1638698805325,detail,755689,True,"749645,753973,755646,486917,420594,757459,7557..."
3012279,1542120650393517,262,763223,1638717116510,detail,763223,True,"749645,753973,755646,486917,420594,757459,7557..."
3015782,1542120650393517,263,761527,1638717310051,detail,761527,True,"749645,753973,755646,486917,420594,757459,7557..."


In [ ]:

typedict2 = {'user_id': int, 'content_id': int, 'desc': object, 'topic_id': int, 'genre_id': int,
             'introduction': object, 'comment': int, 'forward': int, 'thumb': int, 'detail': int, 'neg': int,
             'created_at': int}
df_inter = pd.read_csv('../data/inter_list_final.txt', sep='\t', error_bad_lines=False, encoding='utf-8', header=0,
                       dtype=typedict2)
df_inter1 = pd.merge(df_inter, df2, on='user_id', how='left')

df_inter1['content_num'] = pd.to_numeric(df_inter1['content_num']).fillna('0').astype('int64')

df_inter1 = df_inter1.sort_values(by=['user_id', 'created_at'], ascending=True)
# filter1 = pd.merge(df_inter1, df_content, on='content_id', how='left')
df_inter1.to_csv('../data/dataset/intereaction.txt', sep='\t', encoding='utf-8', index=False)
bt1 = df_inter1[df_inter1['content_num'] > 1]
bt2 = df_inter1[df_inter1['content_num'] > 2]
bt3 = df_inter1[df_inter1['content_num'] > 3]
bt4 = df_inter1[df_inter1['content_num'] > 4]
bt5 = df_inter1[df_inter1['content_num'] > 5]

bt1.to_csv('../data/dataset/bt1.txt', sep='\t', encoding='utf-8', index=False)
bt2.to_csv('../data/dataset/bt2.txt', sep='\t', encoding='utf-8', index=False)
bt3.to_csv('../data/dataset/bt3.txt', sep='\t', encoding='utf-8', index=False)
bt4.to_csv('../data/dataset/bt4.txt', sep='\t', encoding='utf-8', index=False)
bt5.to_csv('../data/dataset/bt5.txt', sep='\t', encoding='utf-8', index=False)


def lent(data):
    try:
        list = data.split(",")
        return len(list)
    except AttributeError:
        return 0


df2['content_num'] = df2['content_list'].apply(lent)

df3 = df2.drop_duplicates(subset=['user_id'], keep='first')

df4 = df3.drop(labels=['event_id', 'event_data', 'follow', 'content_id', 'rep', 'created_at'], axis=1)


def split_dataset(data):
    train_ratio = 0.8
    dev_ratio = 0.1
    train_length = int(len(data) * train_ratio)
    dev_length = int(len(data) * dev_ratio)
    train = data[:train_length]
    train = train.drop(
        labels=['content_list', 'content_num'], axis=1)
    dev = data[train_length:train_length + dev_length]
    dev = dev.drop(
        labels=['content_list', 'content_num'], axis=1)
    test = data[train_length + dev_length:]
    test = test.drop(
        labels=['content_id', 'desc', 'topic_id', 'genre_id', 'introduction', 'comment', 'forward', 'thumb', 'detail',
                'neg', 'created_at', 'content_num'], axis=1)
    return train, dev, test


train_bt1, dev_bt1, test_bt1 = split_dataset(bt1)
train_bt1 = train_bt1.drop(train_bt1[train_bt1['user_id'] == train_bt1.iloc[-1]['user_id']].index)
dev_bt1 = dev_bt1.drop(dev_bt1[dev_bt1['user_id'] == dev_bt1.iloc[0]['user_id']].index)
test_bt1 = test_bt1.drop(test_bt1[test_bt1['user_id'] == test_bt1.iloc[0]['user_id']].index)

train_bt2, dev_bt2, test_bt2 = split_dataset(bt2)
train_bt2 = train_bt2.drop(train_bt2[train_bt2['user_id'] == train_bt2.iloc[-1]['user_id']].index)
dev_bt2 = dev_bt2.drop(dev_bt2[dev_bt2['user_id'] == dev_bt2.iloc[0]['user_id']].index)
test_bt2 = test_bt2.drop(test_bt2[test_bt2['user_id'] == test_bt2.iloc[0]['user_id']].index)

train_bt3, dev_bt3, test_bt3 = split_dataset(bt3)
train_bt3 = train_bt3.drop(train_bt3[train_bt3['user_id'] == train_bt3.iloc[-1]['user_id']].index)
dev_bt3 = dev_bt3.drop(dev_bt3[dev_bt3['user_id'] == dev_bt3.iloc[0]['user_id']].index)
test_bt3 = test_bt3.drop(test_bt3[test_bt3['user_id'] == test_bt3.iloc[0]['user_id']].index)

train_bt4, dev_bt4, test_bt4 = split_dataset(bt4)
train_bt4 = train_bt4.drop(train_bt4[train_bt4['user_id'] == train_bt4.iloc[-1]['user_id']].index)
dev_bt4 = dev_bt4.drop(dev_bt4[dev_bt4['user_id'] == dev_bt4.iloc[0]['user_id']].index)
test_bt4 = test_bt4.drop(test_bt4[test_bt4['user_id'] == test_bt4.iloc[0]['user_id']].index)

train_bt5, dev_bt5, test_bt5 = split_dataset(bt5)
train_bt5 = train_bt5.drop(train_bt5[train_bt5['user_id'] == train_bt5.iloc[-1]['user_id']].index)
dev_bt5 = dev_bt5.drop(dev_bt5[dev_bt5['user_id'] == dev_bt5.iloc[0]['user_id']].index)
test_bt5 = test_bt5.drop(test_bt5[test_bt5['user_id'] == test_bt5.iloc[0]['user_id']].index)

train_bt1.to_csv('../data/dataset/bt1/train.txt', sep='\t', encoding='utf-8', index=False)
dev_bt1.to_csv('../data/dataset/bt1/dev.txt', sep='\t', encoding='utf-8', index=False)
test_bt1.to_csv('../data/dataset/bt1/test.txt', sep='\t', encoding='utf-8', index=False)
train_bt2.to_csv('../data/dataset/bt2/train.txt', sep='\t', encoding='utf-8', index=False)
dev_bt2.to_csv('../data/dataset/bt2/dev.txt', sep='\t', encoding='utf-8', index=False)
test_bt2.to_csv('../data/dataset/bt2/test.txt', sep='\t', encoding='utf-8', index=False)
train_bt3.to_csv('../data/dataset/bt3/train.txt', sep='\t', encoding='utf-8', index=False)
dev_bt3.to_csv('../data/dataset/bt3/dev.txt', sep='\t', encoding='utf-8', index=False)
test_bt3.to_csv('../data/dataset/bt3/test.txt', sep='\t', encoding='utf-8', index=False)
train_bt4.to_csv('../data/dataset/bt4/train.txt', sep='\t', encoding='utf-8', index=False)
dev_bt4.to_csv('../data/dataset/bt4/dev.txt', sep='\t', encoding='utf-8', index=False)
test_bt4.to_csv('../data/dataset/bt4/test.txt', sep='\t', encoding='utf-8', index=False)
train_bt5.to_csv('../data/dataset/bt5/train.txt', sep='\t', encoding='utf-8', index=False)
dev_bt5.to_csv('../data/dataset/bt5/dev.txt', sep='\t', encoding='utf-8', index=False)
test_bt5.to_csv('../data/dataset/bt5/test.txt', sep='\t', encoding='utf-8', index=False)